# EE359-Coursework 4  Streaming Algorithm
Existing codes in this file are just hints. You can modify these codes as you want

## **Task1：DGIM**

DGIM is an efficient algorithm in processing large streams. When it's infeasible to store the flowing binary stream, DGIM can estimate the number of 1-bits in the window. In this coding, you're given the stream_data_dgim.txt (binary stream), and you need to implement the DGIM algorithm to count the number of 1-bits. Write code below.

### 1. Set the window size to 1000, and count the number of 1-bits in the current window.

In [73]:
from collections import defaultdict
import time

class DGIM:
    def __init__(self, filepath, windowsize, maxtime = None):
        self.fileHandler = open(filepath, 'r')
        self.windowSize = windowsize
        self.buckets = defaultdict(list)
        self.timeMod = maxtime if maxtime else windowsize << 2
        self.timestamp = 0
    
    def update(self):
        keys = sorted(self.buckets.keys(), reverse=True)
        for key in keys:
            if len(self.buckets[key]) > 0:
                if min(self.buckets[key]) < self.timestamp - self.windowSize:
                    self.buckets[key].remove(min(self.buckets[key]))
                    break

        
        self.merge_buckets()

    def merge_buckets(self):
        i = 1
        while i < self.timeMod:
            if len(self.buckets[i]) > 2:
                min_1 = min(self.buckets[i])
                self.buckets[i].remove(min_1)
                min_2 = min(self.buckets[i])
                self.buckets[i].remove(min_2)
                self.buckets[i*2].append(min(min_1, min_2))
            i *= 2
    
    def run(self):
        f = self.fileHandler
        x = f.read(2).strip()
        while x:
            if x == '1':
                self.buckets[1].append(self.timestamp)
                self.update()
            self.timestamp = (self.timestamp + 1) % self.timeMod
            
            x = f.read(2).strip()
    
    def count(self, start):
        total = 0
        last_bucket_size = 0 

        for key in sorted(self.buckets.keys()):
            
            if len(self.buckets[key]) == 0:
                continue
            else:
                last_bucket_size = 0

            for _stp in self.buckets[key]:

                if _stp <= self.timestamp - start:
                    continue
                total += key
                last_bucket_size += key

        return total - last_bucket_size // 2


Solver = DGIM(filepath='./stream_data_dgim.txt', windowsize=1000, maxtime=40001)
Solver.run()
start_time = time.time()
predictedCount= Solver.count(start=1000)
end_time = time.time()
pre_running_time = end_time - start_time
print('Predicted Count (last 1000):', predictedCount)

Predicted Count (last 1000): 316
Running Time: 6.818771362304688e-05


### 2. With the window size 1000, count the number of 1-bits in the last 500 and 200 bits of the bitstream.

In [59]:
# code here
print('Predicted Count (last 500):', Solver.count(start=500))

print('Predicted Count (last 200):', Solver.count(start=200))

Predicted Count (last 500): 188
Predicted Count (last 200): 60


### 3. Write a function that accurately counts the number of 1-bits in the current window. Caculate the accuracy of your own DGIM algorithm and compare the running time difference.

In [74]:
# Your code here, you can add cells if necessary
import numpy as np
import time
def accurateCountTask1(filepath='./stream_data_dgim.txt', window_size=1000):
    start_time = time.time()
    stream = []
    with open(filepath, 'r') as f:
        x = f.read(2).strip()
        while x:
            stream.append(int(x))
            x = f.read(2).strip()
    count = sum(stream[-1 * window_size:])
    end_time = time.time()
    return count, end_time - start_time

accurateCount, acc_running_time = accurateCountTask1()
    
print('Accurate Count (last 1000):', accurateCount)
print('Accurancy:', 1 - abs(predictedCount - accurateCount) / accurateCount)
print(f'Running time of DGIM is {1000 * pre_running_time} ms. Running time of accurately counting is {1000* acc_running_time} ms')

Accurate Count (last 1000): 391
Accurancy: 0.8081841432225064
Running time of DGIM is 0.06818771362304688 ms. Running time of accurately counting is 13.064384460449219 ms


## **Task2: Bloom Filter**

A Bloom filter is a space-efficient probabilistic data structure. Here the task is to implement a bloom filter by yourself. 

### Data loading:

From the NLTK (Natural Language ToolKit) library, we import a large list of English dictionary words, commonly used by the very first spell-checking programs in Unix-like operating systems.

In [11]:
import nltk
from nltk.corpus import words
# nltk.download('words')
word_list = words.words()

Then we load another dataset from the NLTK Corpora collection: movie_reviews.

The movie reviews are categorized between positive and negative, so we construct a list of words (usually called bag of words) for each category.

In [14]:
# nltk.download('movie_reviews')
from nltk.corpus import movie_reviews

neg_reviews = []
pos_reviews = []

for fileid in movie_reviews.fileids('neg'):
  neg_reviews.extend(movie_reviews.words(fileid))
for fileid in movie_reviews.fileids('pos'):
  pos_reviews.extend(movie_reviews.words(fileid))

Here we get a data stream (word_list) and 2 query lists (neg_reviews and pos_reviews).

### 1. Write a function that accurately determines whether each word in neg_reviews and pos_reviews belongs to word_list.

In [85]:
import numpy as np
import time

def linear_search(word_list, target_word):
    ## According to the annotation in the next block, I implement the binary search function instead of linear search.
    start = 0
    end = len(word_list) - 1
    mid = 0
    while start <= end:
        mid = (start + end) // 2
    
        if word_list[mid] < target_word:
            start = mid + 1

        elif word_list[mid] > target_word:
            end = mid - 1

        else:
            return True
    
    return False

In [86]:
# Binary search
neg_flag_b = np.zeros(len(neg_reviews), dtype=bool)
pos_flag_b = np.zeros(len(pos_reviews), dtype=bool)
# to store the accurate result

t = time.time()
sorted_word_list = sorted(word_list)
for i, neg_word in enumerate(neg_reviews):
    neg_flag_b[i] = linear_search(sorted_word_list, neg_word)
for i, pos_word in enumerate(pos_reviews):
    pos_flag_b[i] = linear_search(sorted_word_list, pos_word)
print("Search Time:", time.time() - t)

Search Time: 5.356951951980591


 ### 2. Implement the bloom filter by yourself and add all words in word_list in your bloom filter. Compare the running time difference between exact search (Task2 Question1) and multiple hash computations in a Bloom filter.

In [89]:
# Your code here, you can add cells if necessary
import hashlib

class BloomFilter:
    def __init__(self, m, k):
        self.m = int(m)
        self.k = int(k)
        self.table = np.zeros((self.m), dtype = bool)
    def add(self, item):
        hash1 = hashlib.sha256()
        hash1.update(item.encode())
        hash1 = int(hash1.hexdigest(), 16) % self.m
        hash2 = hashlib.md5()
        hash2.update(item.encode())
        hash2 = int(hash2.hexdigest(), 16) % self.m
        h = hash1
        for i in range(self.k):
            self.table[(h + i * hash2) % self.m] = True
    def check(self, item):
        hash1 = hashlib.sha256()
        hash1.update(item.encode())
        hash1 = int(hash1.hexdigest(), 16) % self.m
        hash2 = hashlib.md5()
        hash2.update(item.encode())
        hash2 = int(hash2.hexdigest(), 16) % self.m
        h = hash1
        for i in range(self.k):
            if self.table[(h + i * hash2) % self.m] == False:
                return False
        return True

In [120]:
bf = BloomFilter(1e7, 4)
neg_bf = np.zeros(len(neg_reviews), dtype=bool)
pos_bf = np.zeros(len(pos_reviews), dtype=bool)
n = neg_bf.shape[0] + pos_bf.shape[0]
t = time.time()
for word in word_list:
    bf.add(word)

for i, neg_word in enumerate(neg_reviews):
    neg_bf[i] = bf.check(neg_word)
for i, pos_word in enumerate(pos_reviews):
    pos_bf[i] = bf.check(pos_word)

neg_FP = sum(neg_bf[i] == True and neg_flag_b[i] == False for i in range(len(neg_reviews)))
neg_TN = sum(neg_bf[i] == False and neg_flag_b[i] == False for i in range(len(neg_reviews)))
pos_FP = sum(pos_bf[i] == True and pos_flag_b[i] == False for i in range(len(pos_reviews)))
pos_TN = sum(pos_bf[i] == False and pos_flag_b[i] == False for i in range(len(pos_reviews)))

print("*" * 20)
print("m = 1e7, k = 4")
print("Bloom Filter Time:", time.time()-t)
print("Accuracy:")
print(f"neg: {(neg_bf == neg_flag_b).sum()} / {neg_bf.shape[0]}")
print(f"pos: {(pos_bf == pos_flag_b).sum()} / {pos_bf.shape[0]}")
print(f"False Positive Rate (neg): {neg_FP / (neg_FP + neg_TN)}")
print(f"False Positive Rate (pos): {pos_FP / (pos_FP + pos_TN)}")

********************
m = 1e7, k = 4
Bloom Filter Time: 20.547614336013794
Accuracy:
neg: 751255 / 751256
pos: 832564 / 832564
False Positive Rate (neg): 4.75606160050985e-06
False Positive Rate (pos): 0.0


### 3. Use different bit array length ‘m’ and number of hash functions ‘k’ to implement the bloom filter algorithm. Then compare the impact of different m and k on the false positive rate.

In [121]:
##########################################   m = 1e6, k = 4   ########################################## 

bf = BloomFilter(1e6, 4)
neg_bf = np.zeros(len(neg_reviews), dtype=bool)
pos_bf = np.zeros(len(pos_reviews), dtype=bool)
n = neg_bf.shape[0] + pos_bf.shape[0]
t = time.time()
for word in word_list:
    bf.add(word)

for i, neg_word in enumerate(neg_reviews):
    neg_bf[i] = bf.check(neg_word)
for i, pos_word in enumerate(pos_reviews):
    pos_bf[i] = bf.check(pos_word)

neg_FP = sum(neg_bf[i] == True and neg_flag_b[i] == False for i in range(len(neg_reviews)))
neg_TN = sum(neg_bf[i] == False and neg_flag_b[i] == False for i in range(len(neg_reviews)))
pos_FP = sum(pos_bf[i] == True and pos_flag_b[i] == False for i in range(len(pos_reviews)))
pos_TN = sum(pos_bf[i] == False and pos_flag_b[i] == False for i in range(len(pos_reviews)))

print("*" * 20)
print("m = 1e6, k = 4")
print("Bloom Filter Time:", time.time()-t)
print("Accuracy:")
print(f"neg: {(neg_bf == neg_flag_b).sum()} / {neg_bf.shape[0]}")
print(f"pos: {(pos_bf == pos_flag_b).sum()} / {pos_bf.shape[0]}")
print(f"False Positive Rate (neg): {neg_FP / (neg_FP + neg_TN)}")
print(f"False Positive Rate (pos): {pos_FP / (pos_FP + pos_TN)}")

##########################################   m = 1e5, k = 4   ########################################## 
bf = BloomFilter(1e5, 4)
neg_bf = np.zeros(len(neg_reviews), dtype=bool)
pos_bf = np.zeros(len(pos_reviews), dtype=bool)
n = neg_bf.shape[0] + pos_bf.shape[0]
t = time.time()
for word in word_list:
    bf.add(word)

for i, neg_word in enumerate(neg_reviews):
    neg_bf[i] = bf.check(neg_word)
for i, pos_word in enumerate(pos_reviews):
    pos_bf[i] = bf.check(pos_word)
    
neg_FP = sum(neg_bf[i] == True and neg_flag_b[i] == False for i in range(len(neg_reviews)))
neg_TN = sum(neg_bf[i] == False and neg_flag_b[i] == False for i in range(len(neg_reviews)))
pos_FP = sum(pos_bf[i] == True and pos_flag_b[i] == False for i in range(len(pos_reviews)))
pos_TN = sum(pos_bf[i] == False and pos_flag_b[i] == False for i in range(len(pos_reviews)))

print("*" * 20)
print("m = 1e5, k = 4")
print("Bloom Filter Time:", time.time()-t)
print("Accuracy:")
print(f"neg: {(neg_bf == neg_flag_b).sum()} / {neg_bf.shape[0]}")
print(f"pos: {(pos_bf == pos_flag_b).sum()} / {pos_bf.shape[0]}")
print(f"False Positive Rate (neg): {neg_FP / (neg_FP + neg_TN)}")
print(f"False Positive Rate (pos): {pos_FP / (pos_FP + pos_TN)}")

##########################################   m = 1e4, k = 4   ########################################## 
bf = BloomFilter(1e4, 4)
neg_bf = np.zeros(len(neg_reviews), dtype=bool)
pos_bf = np.zeros(len(pos_reviews), dtype=bool)
n = neg_bf.shape[0] + pos_bf.shape[0]
t = time.time()
for word in word_list:
    bf.add(word)

for i, neg_word in enumerate(neg_reviews):
    neg_bf[i] = bf.check(neg_word)
for i, pos_word in enumerate(pos_reviews):
    pos_bf[i] = bf.check(pos_word)

neg_FP = sum(neg_bf[i] == True and neg_flag_b[i] == False for i in range(len(neg_reviews)))
neg_TN = sum(neg_bf[i] == False and neg_flag_b[i] == False for i in range(len(neg_reviews)))
pos_FP = sum(pos_bf[i] == True and pos_flag_b[i] == False for i in range(len(pos_reviews)))
pos_TN = sum(pos_bf[i] == False and pos_flag_b[i] == False for i in range(len(pos_reviews)))

print("*" * 20)
print("m = 1e4, k = 4")
print("Bloom Filter Time:", time.time()-t)
print("Accuracy:")
print(f"neg: {(neg_bf == neg_flag_b).sum()} / {neg_bf.shape[0]}")
print(f"pos: {(pos_bf == pos_flag_b).sum()} / {pos_bf.shape[0]}")
print(f"False Positive Rate (neg): {neg_FP / (neg_FP + neg_TN)}")
print(f"False Positive Rate (pos): {pos_FP / (pos_FP + pos_TN)}")



********************
m = 1e6, k = 4
Bloom Filter Time: 19.902321577072144
Accuracy:
neg: 695661 / 751256
pos: 767160 / 832564
False Positive Rate (neg): 0.2644132446803451
False Positive Rate (pos): 0.28138498339327817
********************
m = 1e5, k = 4
Bloom Filter Time: 19.596858978271484
Accuracy:
neg: 541011 / 751256
pos: 600144 / 832564
False Positive Rate (neg): 0.9999381711991934
False Positive Rate (pos): 0.9999311638472526
********************
m = 1e4, k = 4
Bloom Filter Time: 19.76619577407837
Accuracy:
neg: 540998 / 751256
pos: 600128 / 832564
False Positive Rate (neg): 1.0
False Positive Rate (pos): 1.0


In [124]:
##########################################   m = 1e7, k = 10   ########################################## 
bf = BloomFilter(1e7, 10)
neg_bf = np.zeros(len(neg_reviews), dtype=bool)
pos_bf = np.zeros(len(pos_reviews), dtype=bool)
n = neg_bf.shape[0] + pos_bf.shape[0]
t = time.time()
for word in word_list:
    bf.add(word)

for i, neg_word in enumerate(neg_reviews):
    neg_bf[i] = bf.check(neg_word)
for i, pos_word in enumerate(pos_reviews):
    pos_bf[i] = bf.check(pos_word)

neg_FP = sum(neg_bf[i] == True and neg_flag_b[i] == False for i in range(len(neg_reviews)))
neg_TN = sum(neg_bf[i] == False and neg_flag_b[i] == False for i in range(len(neg_reviews)))
pos_FP = sum(pos_bf[i] == True and pos_flag_b[i] == False for i in range(len(pos_reviews)))
pos_TN = sum(pos_bf[i] == False and pos_flag_b[i] == False for i in range(len(pos_reviews)))

print("*" * 20)
print("m = 1e7, k = 10")
print("Bloom Filter Time:", time.time()-t)
print("Accuracy:")
print(f"neg: {(neg_bf == neg_flag_b).sum()} / {neg_bf.shape[0]}")
print(f"pos: {(pos_bf == pos_flag_b).sum()} / {pos_bf.shape[0]}")
print(f"False Positive Rate (neg): {neg_FP / (neg_FP + neg_TN)}")
print(f"False Positive Rate (pos): {pos_FP / (pos_FP + pos_TN)}")

##########################################   m = 1e7, k = 5   ########################################## 
bf = BloomFilter(1e7, 5)
neg_bf = np.zeros(len(neg_reviews), dtype=bool)
pos_bf = np.zeros(len(pos_reviews), dtype=bool)
n = neg_bf.shape[0] + pos_bf.shape[0]
t = time.time()
for word in word_list:
    bf.add(word)

for i, neg_word in enumerate(neg_reviews):
    neg_bf[i] = bf.check(neg_word)
for i, pos_word in enumerate(pos_reviews):
    pos_bf[i] = bf.check(pos_word)

neg_FP = sum(neg_bf[i] == True and neg_flag_b[i] == False for i in range(len(neg_reviews)))
neg_TN = sum(neg_bf[i] == False and neg_flag_b[i] == False for i in range(len(neg_reviews)))
pos_FP = sum(pos_bf[i] == True and pos_flag_b[i] == False for i in range(len(pos_reviews)))
pos_TN = sum(pos_bf[i] == False and pos_flag_b[i] == False for i in range(len(pos_reviews)))

print("*" * 20)
print("m = 1e7, k = 5")
print("Bloom Filter Time:", time.time()-t)
print("Accuracy:")
print(f"neg: {(neg_bf == neg_flag_b).sum()} / {neg_bf.shape[0]}")
print(f"pos: {(pos_bf == pos_flag_b).sum()} / {pos_bf.shape[0]}")
print(f"False Positive Rate (neg): {neg_FP / (neg_FP + neg_TN)}")
print(f"False Positive Rate (pos): {pos_FP / (pos_FP + pos_TN)}")

##########################################   m = 1e7, k = 3   ########################################## 
bf = BloomFilter(1e7, 3)
neg_bf = np.zeros(len(neg_reviews), dtype=bool)
pos_bf = np.zeros(len(pos_reviews), dtype=bool)
n = neg_bf.shape[0] + pos_bf.shape[0]
t = time.time()
for word in word_list:
    bf.add(word)

for i, neg_word in enumerate(neg_reviews):
    neg_bf[i] = bf.check(neg_word)
for i, pos_word in enumerate(pos_reviews):
    pos_bf[i] = bf.check(pos_word)

neg_FP = sum(neg_bf[i] == True and neg_flag_b[i] == False for i in range(len(neg_reviews)))
neg_TN = sum(neg_bf[i] == False and neg_flag_b[i] == False for i in range(len(neg_reviews)))
pos_FP = sum(pos_bf[i] == True and pos_flag_b[i] == False for i in range(len(pos_reviews)))
pos_TN = sum(pos_bf[i] == False and pos_flag_b[i] == False for i in range(len(pos_reviews)))

print("*" * 20)
print("m = 1e7, k = 3")
print("Bloom Filter Time:", time.time()-t)
print("Accuracy:")
print(f"neg: {(neg_bf == neg_flag_b).sum()} / {neg_bf.shape[0]}")
print(f"pos: {(pos_bf == pos_flag_b).sum()} / {pos_bf.shape[0]}")
print(f"False Positive Rate (neg): {neg_FP / (neg_FP + neg_TN)}")
print(f"False Positive Rate (pos): {pos_FP / (pos_FP + pos_TN)}")

##########################################   m = 1e7, k = 2   ########################################## 

bf = BloomFilter(1e7, 2)
neg_bf = np.zeros(len(neg_reviews), dtype=bool)
pos_bf = np.zeros(len(pos_reviews), dtype=bool)
n = neg_bf.shape[0] + pos_bf.shape[0]
t = time.time()
for word in word_list:
    bf.add(word)

for i, neg_word in enumerate(neg_reviews):
    neg_bf[i] = bf.check(neg_word)
for i, pos_word in enumerate(pos_reviews):
    pos_bf[i] = bf.check(pos_word)

neg_FP = sum(neg_bf[i] == True and neg_flag_b[i] == False for i in range(len(neg_reviews)))
neg_TN = sum(neg_bf[i] == False and neg_flag_b[i] == False for i in range(len(neg_reviews)))
pos_FP = sum(pos_bf[i] == True and pos_flag_b[i] == False for i in range(len(pos_reviews)))
pos_TN = sum(pos_bf[i] == False and pos_flag_b[i] == False for i in range(len(pos_reviews)))

print("*" * 20)
print("m = 1e7, k = 2")
print("Bloom Filter Time:", time.time()-t)
print("Accuracy:")
print(f"neg: {(neg_bf == neg_flag_b).sum()} / {neg_bf.shape[0]}")
print(f"pos: {(pos_bf == pos_flag_b).sum()} / {pos_bf.shape[0]}")
print(f"False Positive Rate (neg): {neg_FP / (neg_FP + neg_TN)}")
print(f"False Positive Rate (pos): {pos_FP / (pos_FP + pos_TN)}")

##########################################   m = 1e7, k = 1   ########################################## 

bf = BloomFilter(1e7, 1)
neg_bf = np.zeros(len(neg_reviews), dtype=bool)
pos_bf = np.zeros(len(pos_reviews), dtype=bool)
n = neg_bf.shape[0] + pos_bf.shape[0]
t = time.time()
for word in word_list:
    bf.add(word)

for i, neg_word in enumerate(neg_reviews):
    neg_bf[i] = bf.check(neg_word)
for i, pos_word in enumerate(pos_reviews):
    pos_bf[i] = bf.check(pos_word)

neg_FP = sum(neg_bf[i] == True and neg_flag_b[i] == False for i in range(len(neg_reviews)))
neg_N = sum(neg_flag_b[i] == False for i in range(len(neg_reviews)))
pos_FP = sum(pos_bf[i] == True and pos_flag_b[i] == False for i in range(len(pos_reviews)))
pos_N = sum(pos_flag_b[i] == False for i in range(len(pos_reviews)))

print("*" * 20)
print("m = 1e7, k = 1")
print("Bloom Filter Time:", time.time()-t)
print("Accuracy:")
print(f"neg: {(neg_bf == neg_flag_b).sum()} / {neg_bf.shape[0]}")
print(f"pos: {(pos_bf == pos_flag_b).sum()} / {pos_bf.shape[0]}")
print(f"False Positive Rate (neg): {neg_FP / neg_N}")
print(f"False Positive Rate (pos): {pos_FP / pos_N}")

********************
m = 1e7, k = 10
Bloom Filter Time: 27.79302144050598
Accuracy:
neg: 751256 / 751256
pos: 832564 / 832564
False Positive Rate (neg): 0.0
False Positive Rate (pos): 0.0
********************
m = 1e7, k = 5
Bloom Filter Time: 21.780128002166748
Accuracy:
neg: 751256 / 751256
pos: 832564 / 832564
False Positive Rate (neg): 0.0
False Positive Rate (pos): 0.0
********************
m = 1e7, k = 3
Bloom Filter Time: 20.07935667037964
Accuracy:
neg: 751247 / 751256
pos: 832543 / 832564
False Positive Rate (neg): 4.280455440458865e-05
False Positive Rate (pos): 9.034745048099262e-05
********************
m = 1e7, k = 2
Bloom Filter Time: 18.719287872314453
Accuracy:
neg: 751176 / 751256
pos: 832456 / 832564
False Positive Rate (neg): 0.000380484928040788
False Positive Rate (pos): 0.0004646440310451049
********************
m = 1e7, k = 1
Bloom Filter Time: 16.598642826080322
Accuracy:
neg: 747688 / 751256
pos: 828475 / 832564
False Positive Rate (neg): 0.016969627790619144
Fals

## **Task3: Count-Min sketch**



In computing, the count–min sketch (CM sketch) is a probabilistic data structure that serves as a frequency table of events in a stream of data. 

Here we use the query stream (neg_reviews or pos_reviews) from task 2.

### 1. Write a function that accurately counts the occurrence times of each word in neg_reviews or pos_reviews.

In [100]:
# Your code here, you can add cells if necessary

def accCount(words):
    count_dic = {}
    for word in words:
        if word not in count_dic:
            count_dic[word] = 1
        else:
            count_dic[word] += 1
    return count_dic

neg_accCount = accCount(neg_reviews)
pos_accCount = accCount(pos_reviews)

### 2. Implement the Count-Min sketch by yourself. Set different width w and depth d of the internal data structure of CM-Sketch. Compare the influence of different w and d on the error.

In [104]:
# Your code here, you can add cells if necessary
class CountMin:
    def __init__(self, w, d):
        self.w = int(w)
        self.d = int(d)
        self.table=np.zeros((self.d, self.w), dtype=int)
        
    def add(self, item):
        hash1 = hashlib.sha256()
        hash1.update(item.encode())
        hash1 = int(hash1.hexdigest(), 16) % self.w
        hash2 = hashlib.md5()
        hash2.update(item.encode())
        hash2 = int(hash2.hexdigest(), 16) % self.w
        h = hash1
        for i in range(self.d):
            self.table[i, (h + i * hash2) % self.w] += 1

    def count(self, item):
        hash1 = hashlib.sha256()
        hash1.update(item.encode())
        hash1 = int(hash1.hexdigest(), 16) % self.w
        hash2 = hashlib.md5()
        hash2.update(item.encode())
        hash2 = int(hash2.hexdigest(), 16) % self.w
        h = hash1
        return min(self.table[i, (h + i * hash2) % self.w] for i in range(self.d))
    


In [125]:
## define the error computing function
def mean_absolute_error(actual, predicted):
    total_error = 0.0
    count = 0
    for key in actual.keys():
        if key in predicted:
            total_error += abs(actual[key] - predicted[key])
        else:
            total_error += actual[key]
        count += actual[key]

    for key in predicted.keys():
        if key not in actual:
            total_error += predicted[key]

    return int(total_error), count


CM_Sketch = CountMin(w=1e7, d=4)
neg_CMCount = {}

for word in neg_reviews:
    CM_Sketch.add(word)

for neg_word in neg_reviews:
    neg_CMCount[neg_word] = CM_Sketch.count(neg_word)

total_error, count = mean_absolute_error(neg_accCount, neg_CMCount)

print("*" * 20)
print("w = 1e7, d = 4")
print("Mean Absolute Error: {} / {}\n".format(total_error, count))

CM_Sketch = CountMin(w=1e6, d=4)
neg_CMCount = {}

for word in neg_reviews:
    CM_Sketch.add(word)

for neg_word in neg_reviews:
    neg_CMCount[neg_word] = CM_Sketch.count(neg_word)

total_error, count = mean_absolute_error(neg_accCount, neg_CMCount)

print("*" * 20)
print("w = 1e6, d = 4")
print("Mean Absolute Error: {} / {}\n".format(total_error, count))

CM_Sketch = CountMin(w=1e5, d=4)
neg_CMCount = {}

for word in neg_reviews:
    CM_Sketch.add(word)

for neg_word in neg_reviews:
    neg_CMCount[neg_word] = CM_Sketch.count(neg_word)

total_error, count = mean_absolute_error(neg_accCount, neg_CMCount)

print("*" * 20)
print("w = 1e5, d = 4")
print("Mean Absolute Error: {} / {}\n".format(total_error, count))

CM_Sketch = CountMin(w=1e4, d=4)
neg_CMCount = {}

for word in neg_reviews:
    CM_Sketch.add(word)

for neg_word in neg_reviews:
    neg_CMCount[neg_word] = CM_Sketch.count(neg_word)

total_error, count = mean_absolute_error(neg_accCount, neg_CMCount)

print("*" * 20)
print("w = 1e4, d = 4")
print("Mean Absolute Error: {} / {}\n".format(total_error, count))

********************
w = 1e7, d = 4
Mean Absolute Error: 0 / 751256
********************
w = 1e6, d = 4
Mean Absolute Error: 0 / 751256

********************
w = 1e5, d = 4
Mean Absolute Error: 137 / 751256

********************
w = 1e4, d = 4
Mean Absolute Error: 104797 / 751256



In [126]:
CM_Sketch = CountMin(w=1e7, d=3)
neg_CMCount = {}

for word in neg_reviews:
    CM_Sketch.add(word)

for neg_word in neg_reviews:
    neg_CMCount[neg_word] = CM_Sketch.count(neg_word)

total_error, count = mean_absolute_error(neg_accCount, neg_CMCount)

print("*" * 20)
print("w = 1e7, d = 3")
print("Mean Absolute Error: {} / {}\n".format(total_error, count))

CM_Sketch = CountMin(w=1e7, d=2)
neg_CMCount = {}

for word in neg_reviews:
    CM_Sketch.add(word)

for neg_word in neg_reviews:
    neg_CMCount[neg_word] = CM_Sketch.count(neg_word)

total_error, count = mean_absolute_error(neg_accCount, neg_CMCount)

print("*" * 20)
print("w = 1e7, d = 2")
print("Mean Absolute Error: {} / {}\n".format(total_error, count))

CM_Sketch = CountMin(w=1e7, d=1)
neg_CMCount = {}

for word in neg_reviews:
    CM_Sketch.add(word)

for neg_word in neg_reviews:
    neg_CMCount[neg_word] = CM_Sketch.count(neg_word)

total_error, count = mean_absolute_error(neg_accCount, neg_CMCount)

print("*" * 20)
print("w = 1e7, d = 1")
print("Mean Absolute Error: {} / {}\n".format(total_error, count))


********************
w = 1e7, d = 3
Mean Absolute Error: 0 / 751256

********************
w = 1e7, d = 2
Mean Absolute Error: 0 / 751256

********************
w = 1e7, d = 1
Mean Absolute Error: 719 / 751256

